# Hr Neural Network Train Test
* stage block 구성 완료
* 학습 가능한지 점검

* toolbox 구성
* 임의로 graphs 형성 (불러오기 하지말고 바로 넘겨 받을 수 있게 설정)
* 학습 및 acc 확인

In [4]:
import sys
sys.path.insert(0,'../')


In [5]:
from deap import base, creator
from deap import tools


import random
from itertools import repeat
from collections import Sequence

# For evaluate function --------------------------
import glob
from easydict import EasyDict

import numpy as np

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn    # for hardware tunning (cudnn.benchmark = True)

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from thop import profile
from thop import clever_format

import logging

# Gray code package
from utils_kyy.utils_graycode_v2 import *

# custom package in utils_kyy
from utils_kyy.utils_graph import load_graph
from utils_kyy.models_hr import RWNN
from utils_kyy.train_validate import train, validate, train_AMP
from utils_kyy.lr_scheduler import LRScheduler
from torchsummary import summary
# -------------------------------------------------

#from apex import amp



#### Evaluate funtion

In [6]:
# stage_pool_path_list ~ graphs 만들어주는 과정 -> indivudal에서 바로 생성하는 걸로!


def evaluate_hr_full_train(individual, args_train, stage_pool_path_list, data_path=None, channels=109, log_file_name=None):  # individual
    
#     # 1) load graph
#     total_graph_path_list = []
#     for i in range(3):
#         temp = glob.glob(stage_pool_path_list[i] + '*.yaml') # sorting 해줘야함
#         temp.sort()
#         total_graph_path_list.append( temp )        
# #         total_graph_path_list.append( glob.glob(stage_pool_path_list[i] + '*.yaml') )
    
    # grape 

    graph_name = []

    # args_train 셋팅에서 graycode 변환이 true 인지 확인
    if args_train.graycode:
        ## Decode 해줘야 !
        gray_len = len(individual)//3
        for i in range(3):
            # list to string
            tmp = ''
            for j in individual[gray_len*i:gray_len*(i+1)]:
                tmp += str(j)

            # sting to binary to num
            graph_name.append(graydecode(int(tmp)))

    else :
        graph_name = individual

    stage_1_graph = load_graph( total_graph_path_list[0][graph_name[0]] )
    stage_2_graph = load_graph( total_graph_path_list[1][graph_name[1]] )
    stage_3_graph = load_graph( total_graph_path_list[2][graph_name[2]] )
    
    graphs = EasyDict({'stage_1': stage_1_graph,
                       'stage_2': stage_2_graph,
                       'stage_3': stage_3_graph
                      })

    # 2) build RWNN
    channels = channels
    NN_model = RWNN(net_type='small', graphs=graphs, channels=channels, num_classes=args_train.num_classes, input_channel=args_train.input_dim)
    NN_model.cuda()

    ###########################
    # Flops 계산 - [Debug] nn.DataParallele (for multi-gpu) 적용 전에 확인.
    ###########################
    input_flops = torch.randn(1, args_train.input_dim, 32, 32).cuda()
    flops, params = profile(NN_model, inputs=(input_flops, ), verbose=False)

    ## Model summary
    #summary(NN_model, input_size=(1, 224, 224))

    # 3) Prepare for train### 일단 꺼보자!
    #NN_model = nn.DataParallel(NN_model)  # for multi-GPU
    NN_model = nn.DataParallel(NN_model, device_ids=[0,1,2,3])
    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.SGD(NN_model.parameters(), args_train.base_lr,
                                momentum=args_train.momentum,
                                weight_decay=args_train.weight_decay)
    
    start_epoch  = 0
    best_prec1 = 0    
    
    cudnn.benchmark = True    # This flag allows you to enable the inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware.  
    
    ###########################
    # Dataset & Dataloader
    ###########################

    # 이미 다운 받아놨으니 download=False
    # 데이터가 없을 경우, 처음에는 download=True 로 설정해놓고 실행해주어야함
    
    if data_path is None :
        data_path = './data'
    
 
    if args_train.data == "CIFAR10" :

        cutout_length = 16  # from nsga-net github
        
        CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]  # from nsga-net github
        CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]
        
        train_transform = transforms.Compose(
            [
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                Cutout(cutout_length),  # from nsga-net github
                transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
            ])

        val_transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
            ])

        train_dataset = torchvision.datasets.CIFAR10(root=data_path, train=True,
                                                download=True, transform=train_transform)

        val_dataset = torchvision.datasets.CIFAR10(root=data_path, train=False,
                                               download=True, transform=val_transform)
        
    else :
        raise Exception("Data Error, Only CIFAR10 allowed for the moment")


    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args_train.batch_size,
                                              shuffle=True, num_workers=args_train.workers)  

    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args_train.batch_size,
                                             shuffle=False, num_workers=args_train.workers)    
    
    ###########################
    # Train
    ###########################
    niters = len(train_loader)
    niters = 1

    lr_scheduler = LRScheduler(optimizer, niters, args_train)  # (default) args.step = [30, 60, 90], args.decay_factor = 0.1, args.power = 2.0
    epoch_ = 0
    
    for epoch in range(start_epoch, args_train.epochs):
        # train for one epoch
        train(train_loader, NN_model, criterion, optimizer, lr_scheduler, epoch, args_train.print_freq, log_file_name)

        # evaluate on validation set
        prec1 = validate(val_loader, NN_model, criterion, epoch, log_file_name)

        # remember best prec@1 and save checkpoint
#         is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        
        epoch_ = epoch

    return (-best_prec1, flops), epoch_  # Min (-val_accuracy, flops) 이므로 val_accuracy(top1)에 - 붙여서 return



In [7]:
from utils_kyy.utils_hr import *

In [8]:

# 예시 생성
grph_ex =make_random_graph_ex(15)

## G matrix 구하기
random.seed(1234)
nds, inds,onds = get_graph_info(grph_ex)
g_mat = get_g_matrix(nds)

from utils_kyy.utils_hr import operation_dictionary
operation_dict=operation_dictionary()

In [9]:
g_mat.shape

(15, 15)

#### encoding g_mat

In [10]:
## g_mat -> 한 차원으로 피는 것 (encoding individual 만드는 것)

nsize =  g_mat.shape[0]
g_encode = []

for idx, col_idx in enumerate(range(1,nsize)):
    row_idx = idx + 1
    g_encode.extend(g_mat[:row_idx,col_idx].tolist())
    
## encoding 된 걸 다시 g_mat 복원

decode_mat = np.ones((nsize,nsize)) 
# to initlize 7 (disconnected)
decode_mat = decode_mat * 7


row_idx = 0
col_idx = 1
for idx , v in enumerate(g_encode):
    decode_mat[row_idx][col_idx] = v
    
    if row_idx + 1  == col_idx :
        col_idx+= 1
        row_idx = 0 
    else :
        row_idx += 1
        
        
def gmat2ind(g_mat):
    nsize =  g_mat.shape[0]
    g_encode = []

    for idx, col_idx in enumerate(range(1,nsize)):
        row_idx = idx + 1
        g_encode.extend(g_mat[:row_idx,col_idx].tolist())

    return g_encode

def ind2gmat(g_encode,nize):
    decode_mat = np.ones((nsize,nsize)) 
    
    # to initlize 7 (disconnected)
    decode_mat = decode_mat * 7


    row_idx = 0
    col_idx = 1
    for idx , v in enumerate(g_encode):
        decode_mat[row_idx][col_idx] = v

        if row_idx + 1  == col_idx :
            col_idx+= 1
            row_idx = 0 
        else :
            row_idx += 1
            
    return decode_mat

In [11]:
g_mat.shape

(15, 15)

In [19]:
## parameters 
args_train = EasyDict({
    
    'data_path' : 'D:/data/',
    
        "data": "CIFAR10",
        "num_classes" : 10,
        
        "graycode" : False,
        "hr" : True,
        
        "input_dim" : 3,
        "lr_mode": "cosine",
        "warmup_mode": "linear",    
        "base_lr": 0.2,
        "momentum": 0.9, 
        "weight_decay": 0.00005,
        "print_freq": 100,
        
        "epochs": 2,
        "batch_size": 80,
        "workers": 0,
        
        "warmup_epochs": 0,
        "warmup_lr": 0.0,
        "targetlr": 0.0,
    
        "nsize" : 25
    
    
    
    
})

data_path = args_train.data_path
channels=109
num_classes=10
input_channel=3

log_file_name = 'temp'

In [13]:

# ind -> graph

individual = gmat2ind(g_mat)

graphs_list = [gmat2graph(g_mat) for i in range(3)]

graphs = EasyDict({'stage_1': graphs_list[0],
                   'stage_2': graphs_list[1],
                   'stage_3': graphs_list[2]
                  })



# ind -> gmat

gmats = [ind2gmat(individual,nsize) for i in range(3)] 

[array([[7., 6., 7., 7., 0., 7., 7., 7., 7., 7., 7., 7., 7., 7., 1.],
        [7., 7., 7., 3., 7., 0., 7., 7., 7., 7., 7., 7., 2., 7., 5.],
        [7., 7., 7., 7., 7., 6., 7., 5., 7., 0., 7., 7., 7., 4., 7.],
        [7., 7., 7., 7., 0., 4., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
        [7., 7., 7., 7., 7., 0., 7., 7., 7., 7., 7., 0., 7., 7., 7.],
        [7., 7., 7., 7., 7., 7., 7., 5., 7., 6., 7., 7., 7., 7., 7.],
        [7., 7., 7., 7., 7., 7., 7., 7., 0., 7., 7., 6., 7., 7., 7.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 2., 7., 7., 7., 3., 7.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 1., 0., 7., 5., 7., 7.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 0., 4., 7., 7.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 3., 1., 7.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 0., 7.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 0.],
        [7., 7., 7.,

In [14]:
len(g_mat)

15

### Check

In [14]:
from utils_kyy.utils_hr import operation_dictionary
operation_dict=operation_dictionary()

class double_unit(nn.Module):
    def __init__(self, inplanes, outplanes, stride=1):
        super(double_unit, self).__init__()
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm2d(outplanes)

    def forward(self, x):
        out = self.relu(x)
        #print("Double RELU output",out.size())
        out = self.bn(out)
        #print("Double BN output",out.size())
        return out
        
class Triplet_unit(nn.Module):
    def __init__(self, inplanes, outplanes, stride=1):
        super(Triplet_unit, self).__init__()
        self.relu = nn.ReLU()
        self.conv = depthwise_separable_conv_3x3(inplanes, outplanes, stride)
        self.bn = nn.BatchNorm2d(outplanes)

    def forward(self, x):
        out = self.relu(x)
        #print("Triplet Relu output",out.size())
        out = self.conv(out)
        #print("Triplet Conv output",out.size())
        out = self.bn(out)
        #print("Triplet BN output",out.size())
        return out

class Node_OP(nn.Module):
    def __init__(self, Node, inplanes, outplanes, gmat):
        super(Node_OP, self).__init__()
        self.is_input_node = Node.type == 0
        self.input_nums = len(Node.inputs)  # 해당 Node에 input으로 연결된 노드의 개수
        
        ## For hierachical representation
        self.id = Node.id
        self.oplist = nn.ModuleList()
        self.gmat = gmat
        self.Node = Node
        
        #print("is it input {}, how many inputs {}".format(self.is_input_node,self.input_nums))
        # get operation from g_matirx
        if self.input_nums >= 1:
            for in_idx in Node.inputs:
                operation_idx = gmat[in_idx][self.id]
                op = operation_dict[operation_idx]
                self.oplist.append(op(nin=outplanes, nout=outplanes, stride=1)) ## input node가 아닐 떄는 channel size 유지!!

        # input 개수가 1보다 크면, 여러 input을 합쳐야함.
        #print("input nums",self.input_nums)
        if self.input_nums >= 1:
            self.mean_weight = nn.Parameter(torch.ones(self.input_nums))  # type: torch.nn.parameter.Parameter
            self.sigmoid = nn.Sigmoid()

        if self.is_input_node:
            self.conv = Triplet_unit(inplanes, outplanes, stride=2) 
        else:
            self.conv = double_unit(outplanes, outplanes, stride=1)

        
    # [참고] nn.Sigmoid()(torch.ones(1)) = 0.7311
    # seoungwonpark source 에서는 torch.zeros()로 들어감. => 0.5
    def forward(self, *input):
        #print("Node ID : {} input nums forward {} is it input node {}".format(self.id,self.input_nums,self.is_input_node))
        #print("Operation list", self.oplist)
        if self.input_nums > 1 and self.is_input_node == False:
            #print('='*10)
            #print("Node ID {} Components mean_weight : {} Operation type : {} input size : {}".format(self.id, self.mean_weight[0].size(),
                                                                                                    # self.oplist[0],input[0].size()))
            out = self.sigmoid(self.mean_weight[0]) * self.oplist[0](input[0])  ## input node가 아니라고 생각해도 됨!
            #print("Node ID : {} Output : {}".format(self.id,out.size()))
            for i in range(1, self.input_nums):
                #print("input index {}, input : {} Operation Type {}".format(i, input[i].size(),self.oplist[i]))
                out = out + self.sigmoid(self.mean_weight[i]) * self.oplist[i](input[i])
                #print("Input Node ID : {} Output : {}".format(id,out.size()))

            #print("Multi input Node output",out)
        elif self.input_nums == 1  and self.is_input_node == False:
            out = self.sigmoid(self.mean_weight[0]) * self.oplist[0](input[0])  ## input node가 아니라고 생각해도 됨!
            #print("Components \n self.mean_weight : {} Operation : {} Sigmoid : {} input : {} ".format(self.mean_weight[0], self.oplist[0],self.sigmoid(self.mean_weight[0]),input[0].size()))

            #print("node id : {} \n after sigmoid Output : {}".format(self.id,out.size()))

        else :
            out = input[0]
            #print("Node ID : {} is a input Node \n Output : {}".format(self.id,out))

        #print("node id : {}, before conv Out put : {}".format(self.id,out))
        out = self.conv(out)
        
        #print('='*50)
        #print("node id : {}, Final Out put : {}".format(self.id,out))
        return out


class StageBlock(nn.Module):
    def __init__(self, graph, inplanes, outplanes, gmat):
        super(StageBlock, self).__init__()
        # graph를 input으로 받아서, Node_OP class. 즉, neural network graph로 전환함.
        self.nodes, self.input_nodes, self.output_nodes = get_graph_info(graph)
        self.nodeop = nn.ModuleList()  # Holds submodules in a list.
        self.gmat = gmat

        for node in self.nodes:
            # 각각의 node들을 Node_OP class로 만들어준 뒤, nn.ModuleList()인 self.nodeop에 append 해주기
            self.nodeop.append(Node_OP(node, inplanes, outplanes, self.gmat))

    def forward(self, x):
        results = {}
        # input
        for id in self.input_nodes:
            results[id] = self.nodeop[id](x)  # input x를 먼저 graph's input node에 각각 넣어줌.
        # graph 중간 계산
        for id, node in enumerate(self.nodes):
            # 각각의 노드 id에 대해
            if id not in self.input_nodes:
                # graph's input node가 아니라면, 그래프 내에서 해당 노드의 인풋들인 node.inputs의 output인 results[_id]
                #    => 그 결과를 results[id]에 저장.
                # self.nodeop[id]는 해당 id의 Node_OP. 즉, input들을 받아서 forward(모아서, conv 태우기)하는 것.
                # 따라서, input으로 넣을 때 unpack 함.
                # id 작은 노드부터 result를 차근차근 계산하면서, id를 올라감.
                #print("input size if id {}".format(id))
                results[id] = self.nodeop[id](*[results[_id] for _id in node.inputs])
                
                ## inspecting None
                #print(self.nodeop[id](*[results[_id] for _id in node.inputs]))
                #print("*** Starting id out : {}, ids in : {} , results : {}  node operation : {}".format(id,node.inputs,results,self.nodeop[id]))
                #print('='*30)
                #print("Is Input node :  {} , Result id : {} Result Output shape : {} nodeop : {} ".format(id in self.input_nodes,
                                                                                     #   id,results[id], self.nodeop[id]))

        result = results[self.output_nodes[0]]
        # output
        # graph's output_nodes의 output 들을 평균내기
        for idx, id in enumerate(self.output_nodes):
            if idx > 0:
                result = result + results[id]
        result = result / len(self.output_nodes)
        return result
    
    
class RWNN(nn.Module):
    def __init__(self, net_type , graphs, gmats, channels, num_classes=1000, input_channel=3):
        super(RWNN, self).__init__()

        self.input_channel = input_channel
        self.gmats = gmats
        # 논문에서도 conv1 쪽은 예외적으로 Conv-BN 이라고 언급함. (나머지에서는 Conv-ReLU-BN 을 conv 로 표기)
        #         self.conv1 = depthwise_separable_conv_3x3(input_channel, channels // 2, 2)    # nin, nout, stride
        self.conv1 = depthwise_separable_conv_3x3(input_channel, channels // 2, 1)  # nin, nout, stride
        self.bn1 = nn.BatchNorm2d(channels // 2)

        # 채널수 변화도, 논문에서처럼 conv2: C, conv3: 2C, conv4: 4C, conv5: 8C
        if net_type == 'small':
            #             self.conv2 = Triplet_unit(channels // 2, channels, 2)    # inplanes, outplanes, stride=2
            self.conv2 = Triplet_unit(channels // 2, channels, 1)  # inplanes, outplanes, stride=1

            #             self.conv3 = StageBlock(graphs.stage_1, channels // 2, channels)
            self.conv3 = StageBlock(graphs.stage_1, channels, channels,self.gmats[0])

            self.conv4 = StageBlock(graphs.stage_2, channels, channels * 2, self.gmats[1])

            self.conv5 = StageBlock(graphs.stage_3, channels * 2, channels * 4, self.gmats[2])

            self.relu = nn.ReLU()

            self.bn2 = nn.BatchNorm2d(channels * 4)
            self.avgpool = nn.AvgPool2d(4, stride=1)  # 마지막은 global average pooling
            self.fc = nn.Linear(channels * 4, num_classes)

        #         if net_type == 'small':
        #             self.conv2 = Triplet_unit(channels // 2, channels, 2)    # inplanes, outplanes, stride=2

        #             self.conv3 = StageBlock(graphs.stage_1, channels, channels)

        #             self.conv4 = StageBlock(graphs.stage_2, channels, channels *2)

        #             self.conv5 = StageBlock(graphs.stage_3, channels * 2, channels * 4)

        #             self.relu = nn.ReLU()
        # #             self.conv = nn.Conv2d(channels * 4, 1280, kernel_size=1)   # 마지막에 1x1 conv, 1280-d
        # #             self.bn2 = nn.BatchNorm2d(1280)

        #######################################
        # 원 코드에서 regular 부분 지움
        #######################################
        #         self.avgpool = nn.AvgPool2d(7, stride=1)  # 마지막은 global average pooling
        #         self.fc = nn.Linear(1280, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))

    def forward(self, x):
        #print("==========Data Size", x.size())
        x = self.conv1(x)
        x = self.bn1(x)
        #print("==========Conv1 Size", x.size())  # => (-, 54, 16, 16)

        x = self.conv2(x)
        #print("==========Conv2  Size", x.size())  # => (-, 109, 8, 8)
        x = self.conv3(x)
        #print("==========Conv3  Size", x.size())  # => (-, 109, 8, 8)

        x = self.conv4(x)
        #         print("==========Conv4  Size", x.size())  # => (-, 218, 4, 4)

        x = self.conv5(x)
        #print("==========Conv5  Size", x.size())  # => (-, 436, 2, 2)

        #         x = self.relu(x)  # => (-, 436, 1, 1)
        #         x = self.conv(x)
        #         print("==========Conv6  Size", x.size())  # => (-, 1280, 1, 1)

        x = self.bn2(x)
        x = self.relu(x)

        #print("==========before avgpool  Size", x.size())   ## [수정] CIFAR-10 에서는 여기까지오면 (-, 1280, 7, 7) 이 아니라 (-, ,1280, 1, 1)
        x = self.avgpool(x)
        #         print("==========after avgpool  Size", x.size())
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [15]:
g_mat[:,1]

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [16]:
# 2) build RWNN
channels = 109
NN_model = RWNN(net_type='small', graphs=graphs, channels=109,gmats=gmats ,num_classes=10, input_channel=3)
NN_model.cuda()

###########################
# Flops 계산 - [Debug] nn.DataParallele (for multi-gpu) 적용 전에 확인.
###########################
input_flops = torch.randn(1, input_channel, 32, 32).cuda()
flops, params = profile(NN_model, inputs=(input_flops, ), verbose=False) ## 사이즈 완료

In [17]:
## Model summary
#summary(NN_model, input_size=(1, 224, 224))

# 3) Prepare for train### 일단 꺼보자!
#NN_model = nn.DataParallel(NN_model)  # for multi-GPU
#NN_model = nn.DataParallel(NN_model, device_ids=[0,1,2,3])

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(NN_model.parameters(), args_train.base_lr,
                            momentum=args_train.momentum,
                            weight_decay=args_train.weight_decay)

start_epoch  = 0
best_prec1 = 0    

cudnn.benchmark = True    # This flag allows you to enable the inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware.  

###########################
# Dataset & Dataloader
###########################

# 이미 다운 받아놨으니 download=False
# 데이터가 없을 경우, 처음에는 download=True 로 설정해놓고 실행해주어야함

if data_path is None :
    data_path = './data'


if args_train.data == "CIFAR10" :

    cutout_length = 16  # from nsga-net github

    CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]  # from nsga-net github
    CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

    train_transform = transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
        ])

    val_transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
        ])

    train_dataset = torchvision.datasets.CIFAR10(root=data_path, train=True,
                                            download=True, transform=train_transform)

    val_dataset = torchvision.datasets.CIFAR10(root=data_path, train=False,
                                           download=True, transform=val_transform)

else :
    raise Exception("Data Error, Only CIFAR10 allowed for the moment")


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args_train.batch_size,
                                          shuffle=True, num_workers=args_train.workers)  

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args_train.batch_size,
                                         shuffle=False, num_workers=args_train.workers)    

###########################
# Train
###########################
niters = len(train_loader)
niters = 1

lr_scheduler = LRScheduler(optimizer, niters, args_train)  # (default) args.step = [30, 60, 90], args.decay_factor = 0.1, args.power = 2.0
epoch_ = 0

#for epoch in range(start_epoch, args_train.epochs):
for epoch in range(start_epoch, 30):

    # train for one epoch
    train(train_loader, NN_model, criterion, optimizer, lr_scheduler, epoch, args_train.print_freq, log_file_name)

    # evaluate on validation set
    prec1 = validate(val_loader, NN_model, criterion, epoch, log_file_name)

    # remember best prec@1 and save checkpoint
#         is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)

    epoch_ = epoch

Files already downloaded and verified
Files already downloaded and verified
	 - Epoch: [0][0/625]	Time 0.819 (0.819)	Loss 2.3352 (2.3352)	Prec@1 12.500 (12.500)	Prec@5 58.750 (58.750)


KeyboardInterrupt: 

* Training 성공 !

### NSGA 적용
* Initialization  - WS으로 Graph 및 initialization 생성
* Crossover - gray encodng 처럼 같은 stage 안에서만
* Mutation - 요거는 생각해보자

In [15]:
from deap import base, creator
from deap import tools


import random
from itertools import repeat
from collections import Sequence

# For evaluate function --------------------------
import glob
from easydict import EasyDict

import numpy as np

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn    # for hardware tunning (cudnn.benchmark = True)

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from thop import profile
from thop import clever_format

import logging

# Gray code package
from utils_kyy.utils_graycode_v2 import *

# custom package in utils_kyy
from utils_kyy.utils_graph import load_graph
from utils_kyy.models_hr import RWNN
from utils_kyy.train_validate import train, validate, train_AMP
from utils_kyy.lr_scheduler import LRScheduler
from torchsummary import summary
# -------------------------------------------------

#from apex import amp



In [16]:
# stage_pool_path_list ~ graphs 만들어주는 과정 -> indivudal에서 바로 생성하는 걸로!


def evaluate_hr_full_train(individual, args_train,data_path=args_train.data_path, channels=109, log_file_name=None):  # individual

    graphs = []
    gmats = []
    
    for ind in individual:
        gmat = ind2gmat(ind, args_train.nsize)
        gmats.append(gmat)
        graphs.append(gmat2graph(gmat))

    
    graphs = EasyDict({'stage_1': graphs[0],
                       'stage_2': graphs[1],
                       'stage_3': graphs[2]
                      })

    # 2) build RWNN
    channels = channels
    NN_model = RWNN(net_type='small', graphs=graphs,gmats=gmats ,channels=channels, num_classes=args_train.num_classes, input_channel=args_train.input_dim)
    NN_model.cuda()

    ###########################
    # Flops 계산 - [Debug] nn.DataParallele (for multi-gpu) 적용 전에 확인.
    ###########################
    input_flops = torch.randn(1, args_train.input_dim, 32, 32).cuda()
    flops, params = profile(NN_model, inputs=(input_flops, ), verbose=False)

    ## Model summary
    #summary(NN_model, input_size=(1, 224, 224))

    # 3) Prepare for train### 일단 꺼보자!
    #NN_model = nn.DataParallel(NN_model)  # for multi-GPU
    #NN_model = nn.DataParallel(NN_model, device_ids=[0,1,2,3])
    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.SGD(NN_model.parameters(), args_train.base_lr,
                                momentum=args_train.momentum,
                                weight_decay=args_train.weight_decay)
    
    start_epoch  = 0
    best_prec1 = 0    
    
    cudnn.benchmark = True    # This flag allows you to enable the inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware.  
    
    ###########################
    # Dataset & Dataloader
    ###########################

    # 이미 다운 받아놨으니 download=False
    # 데이터가 없을 경우, 처음에는 download=True 로 설정해놓고 실행해주어야함
    
    if data_path is None :
        data_path = './data'
    
 
    if args_train.data == "CIFAR10" :

        cutout_length = 16  # from nsga-net github
        
        CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]  # from nsga-net github
        CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]
        
        train_transform = transforms.Compose(
            [
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
              #  Cutout(cutout_length),  # from nsga-net github
                transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
            ])

        val_transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
            ])

        train_dataset = torchvision.datasets.CIFAR10(root=data_path, train=True,
                                                download=True, transform=train_transform)

        val_dataset = torchvision.datasets.CIFAR10(root=data_path, train=False,
                                               download=True, transform=val_transform)
        
    else :
        raise Exception("Data Error, Only CIFAR10 allowed for the moment")


    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args_train.batch_size,
                                              shuffle=True, num_workers=args_train.workers)  

    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args_train.batch_size,
                                             shuffle=False, num_workers=args_train.workers)    
    
    ###########################
    # Train
    ###########################
    niters = len(train_loader)
    niters = 1

    lr_scheduler = LRScheduler(optimizer, niters, args_train)  # (default) args.step = [30, 60, 90], args.decay_factor = 0.1, args.power = 2.0
    epoch_ = 0
    
    for epoch in range(start_epoch, args_train.epochs):
        # train for one epoch
        train(train_loader, NN_model, criterion, optimizer, lr_scheduler, epoch, args_train.print_freq, log_file_name)

        # evaluate on validation set
        prec1 = validate(val_loader, NN_model, criterion, epoch, log_file_name)

        # remember best prec@1 and save checkpoint
#         is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        
        epoch_ = epoch

    return (-best_prec1, flops), epoch_  # Min (-val_accuracy, flops) 이므로 val_accuracy(top1)에 - 붙여서 return



In [17]:
# sample individuals to train

nds , _, _  = get_graph_info(grph_ex)
individuals = [gmat2ind(get_g_matrix(nds)) for i in range(3)]
fitness_value , epoch = evaluate_hr_full_train(individuals,args_train)

Files already downloaded and verified
Files already downloaded and verified
	 - Epoch: [0][0/625]	Time 0.807 (0.807)	Loss 2.3336 (2.3336)	Prec@1 7.500 (7.500)	Prec@5 46.250 (46.250)
	 - Epoch: [0][100/625]	Time 0.168 (0.186)	Loss 1.7982 (2.1401)	Prec@1 28.750 (22.611)	Prec@5 86.250 (75.557)
	 - Epoch: [0][200/625]	Time 0.212 (0.183)	Loss 1.7701 (1.9434)	Prec@1 28.750 (28.619)	Prec@5 86.250 (81.163)
	 - Epoch: [0][300/625]	Time 0.209 (0.183)	Loss 1.5875 (1.8294)	Prec@1 45.000 (32.728)	Prec@5 86.250 (84.115)
	 - Epoch: [0][400/625]	Time 0.170 (0.182)	Loss 1.3567 (1.7453)	Prec@1 53.750 (35.842)	Prec@5 91.250 (86.022)
	 - Epoch: [0][500/625]	Time 0.177 (0.181)	Loss 1.2208 (1.6787)	Prec@1 55.000 (38.363)	Prec@5 98.750 (87.405)
	 - Epoch: [0][600/625]	Time 0.211 (0.182)	Loss 1.1309 (1.6215)	Prec@1 61.250 (40.566)	Prec@5 97.500 (88.471)
##### Validation_time 6.266 Prec@1 54.130 Prec@5 94.610 #####
	 - Epoch: [1][0/625]	Time 0.202 (0.202)	Loss 1.2452 (1.2452)	Prec@1 58.750 (58.750)	Prec@5 95.0

In [18]:
## Check evaluation funtion
print(fitness_value, epoch)

(-65.66, 983578624.0) 1


In [ ]:
## For initializing random Graph WS

def init_graph(nsize):
    # nsize : number of nodes
    
    individuals = []
    
    for i in range(3) : # For s stages
        grph_ex =make_random_graph_ex(nsize) # Make random graph using WS

        ## G matrix 구하기
        nds, inds,onds = get_graph_info(grph_ex)
        g_mat = get_g_matrix(nds)

        individual = gmat2ind(g_mat)
        individuals.extend(individual)
    
    return individuals

In [20]:
## Tool box for hr
# individual 바뀌어야

def create_toolbox_for_NSGA_RWNN_hr(args_train, data_path=None ,log_file_name=None):
    # => Min ( -val_accuracy(top_1),  flops )
    creator.create('FitnessMin', base.Fitness, weights=(-1.0, -1.0 ))  # name, base (class), attribute // 
    creator.create('Individual', list, fitness=creator.FitnessMin)  # creator.FitnessMaxMin attribute로 가짐    
    
    #####################################
    # Initialize the toolbox
    #####################################
    toolbox = base.Toolbox()
    if args_train.graycode :
        gray_len = len(str(grayCode(num_graph-1)))
        IND_SIZE = gray_len * 3
        BOUND_LOW = 0
        BOUND_UP = 1
        toolbox.register('attr_int', random.randint, BOUND_LOW, BOUND_UP)
    
    
    elif args_train.hr :
        ## Nsize -> individual size
        nsize = args_train.nsize
        BOUND_LOW = 0 
        BOUND_UP = 7 # Operation 종류
        #toolbox.register('attr_int', random.randint, BOUND_LOW, BOUND_UP) 
        
        toolbox.register('init_graph',nsize)
        
        ## WS 으로 초기화시키는 함수를 만들자, 
        # input : nsize
        # output : nsize -> Graph -> Gmat -> indivdual로 연계되게
        
        

    else:
        IND_SIZE = 3    # 한 individual, 즉 하나의 chromosome은 3개의 graph. 즉, 3개의 stage를 가짐.

        # toolbox.attribute(0, (num_graph-1)) 이렇게 사용함.
        # 즉, 0 ~ (num_grpah - 1) 중 임의의 정수 선택. => 이걸 3번하면 하나의 small graph가 생김
        BOUND_LOW = 0
        BOUND_UP = num_graph-1
        toolbox.register('attr_int', random.randint, BOUND_LOW, BOUND_UP)   # register(alias, method, argument ...)
        # toolbox.attribute라는 함수를 n번 시행해서 containter인 creator.individual에 넣은 후 해당 instance를 반환함.
        # e.g. [0, 1, 3] 반환
        
    ## HR 일 때는 WS로 initiat 하고 싶은데 이 부분 좀 생각이 필요할듯
    toolbox.register('individual', tools.initRepeat,
                     creator.Individual, toolbox.init_graph, nsize)

    toolbox.register('population', tools.initRepeat,
                     list, toolbox.individual)    # n은 생략함. toolbox.population 함수를 뒤에서 실행할 때 넣어줌.    
    
    # crossover
    if args_train.graycode :
        toolbox.register('mate', cxgray, num_graph=num_graph)
    
    elif args_train.hr :
        toolbox.register('mate',cxhr)
    else:
        toolbox.register('mate', tools.cxTwoPoint)  # crossover

    # mutation
    toolbox.register('mutate', mutUniformInt_custom, low=BOUND_LOW, up=BOUND_UP)

    # selection
    # => return A list of selected individuals.
    toolbox.register('select', tools.selNSGA2, nd='standard')  # selection.  // k – The number of individuals to select. k는 함수 쓸 때 받아야함
    
    #########################
    # Seeding a population - train_log 읽어와서 해당 log의 마지막 population으로 init 후 이어서 train 시작
    #########################
    # [Reference] https://deap.readthedocs.io/en/master/tutorials/basic/part1.html
    def LoadIndividual(icls, content):
        return icls(content)

    def LoadPopulation(pcls, ind_init, last_population):  # list of [chromosome, [-val_accuracy, flops]]
        return pcls(ind_init(last_population[i][0]) for i in range(len(last_population)))

    toolbox.register("individual_load", LoadIndividual, creator.Individual)

    toolbox.register("population_load", LoadPopulation, list, toolbox.individual_load)
    
    return toolbox


### GA function test

In [ ]:

def init_graph(nsize):
    # nsize : number of nodes

    individuals = []

    for i in range(3):  # For s stages
        grph_ex = make_random_graph_ex(nsize)  # Make random graph using WS

        ## G matrix 구하기
        nds, inds, onds = get_graph_info(grph_ex)
        g_mat = get_g_matrix(nds)

        individual = gmat2ind(g_mat)
        individuals.extend(individual)

    return individuals

### Main code

In [ ]:
import os
import sys
import os
import logging
from easydict import EasyDict
import numpy as np
import random
import time
import datetime
from deap import tools
from collections import OrderedDict
from pprint import pprint
import json
import torch

sys.path.insert(0, '../')
from utils_kyy.utils_hr import make_random_graph_ex
from utils_kyy.create_toolbox_hr import create_toolbox_for_NSGA_RWNN_hr, evaluate_hr_full_train


import argparse

class rwns_train:
    def __init__(self, json_file):
        self.root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        self.param_dir = os.path.join(self.root + '/parameters/', json_file)
        f = open(self.param_dir)
        params = json.load(f)
        pprint(params)
        self.name = params['NAME']

        ## toolbox params
        self.args_train = EasyDict(params['ARGS_TRAIN'])
        self.data_path = params['DATA_PATH']
        self.run_code = params['RUN_CODE']
        self.stage_pool_path = '../graph_pool' + '/' + self.run_code + '_' + self.name + '/'
        self.stage_pool_path_list = []
        for i in range(1, 4):
            stage_pool_path_i = self.stage_pool_path + str(i) + '/'  # eg. [graph_pool/run_code_name/1/, ... ]
            self.stage_pool_path_list.append(stage_pool_path_i)
        
        self.log_path = '../logs/' + self.run_code + '_' + self.name + '/'
        # self.log_file_name : Initialize 부터 GA 진행상황 등 코드 전체에 대한 logging
        self.log_file_name = self.log_path + 'logging.log'
        # self.train_log_file_name : fitness (= flops, val_accuracy). 즉 GA history 를 저장 후, 나중에 사용하기 위한 logging.
        self.train_log_file_name = self.log_path + 'train_logging.log'
        
        if not os.path.exists(self.stage_pool_path):
            os.makedirs(self.stage_pool_path)
            for i in range(3):
                os.makedirs(self.stage_pool_path_list[i])
                
        if not os.path.isdir(self.log_path):
            os.makedirs(self.log_path)
            
        logging.basicConfig(filename=self.log_file_name, level=logging.INFO)
        logging.info('[Start] Rwns_train class is initialized.')        
        logging.info('Start to write log.')
            
        self.num_graph = params['NUM_GRAPH']
        
        self.toolbox = self.create_toolbox()

        
        ## GA params
        self.pop_size = params['POP_SIZE']
        self.ngen = params['NGEN']
        self.cxpb = params['CXPB']
        self.mutpb = params['MUTPB']

        ## logs
        self.log = OrderedDict()
        self.log['hp'] = self.args_train
        self.train_log = OrderedDict()
        

    def create_toolbox(self):
        make_random_graph_ex(self.nsize)

        return create_toolbox_for_NSGA_RWNN(self.num_graph, self.args_train, self.data_path, self.log_file_name)


    def train(self):
        ###################################
        # 1. Initialize the population.  (toolbox.population은 creator.Individual n개를 담은 list를 반환. (=> population)
        ###################################
        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("[GA] Initialion starts ...")
        logging.info("[GA] Initialion starts at " + now_str)
        init_start_time = time.time()

        pop = self.toolbox.population(n=self.pop_size)

        # fitness values (= accuracy, flops) 모음
        GA_history_list = []    # (choromosome, accuracy, flops) 이렇게 담아놓기
                                # e.g.  [ [[1,3,4], 40, 1000], [[2,6,10], 30%, 2000], ...  ]
        
        ###################################
        # 2. Evaluate the population (with an invalid fitness)
        ###################################
        invalid_ind = [ind for ind in pop]

        for idx, ind in enumerate(invalid_ind):
            fitness, ind_model = evaluate_v2(ind, args_train=self.args_train, stage_pool_path_list=self.stage_pool_path_list,
                                          data_path=self.data_path, log_file_name=self.log_file_name)
            ind.fitness.values = fitness
            GA_history_list.append([ind, fitness])
            
        ## log 기록 - initialize (= 0th generation)
        self.train_log[0] = GA_history_list
        
        self.save_log()
        
        # This is just to assign the crowding distance to the individuals
        # no actual selection is done
        pop = self.toolbox.select(pop, len(pop))

        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("Initialization is finished at", now_str)
        logging.info("Initialion is finished at " + now_str)

        init_time = time.time() - init_start_time
        logging.info("Initialization time = " + str(init_time) + "s")
        print()
        
        ###################################
        # 3. Begin GA
        ###################################
        # Begin the generational process
        for gen in range(1, self.ngen):
            ##### 3.1. log 기록
            now = datetime.datetime.now()
            now_str = now.strftime('%Y-%m-%d %H:%M:%S')
            print("#####", gen, "th generation starts at", now_str)
            logging.info("#####" + str(gen) + "th generation starts at" + now_str)

            start_gen = time.time()
            
            ##### 3.2. Offspring pool 생성 후, crossover(=mate) & mutation
            # Vary the population
            offspring = tools.selTournamentDCD(pop, len(pop))
            offspring = [self.toolbox.clone(ind) for ind in offspring]
            
            # ::2, 1::2 즉, 짝수번째 크로모좀과 홀수번쨰 크로모좀들 차례로 선택하면서 cx, mut 적용
            # e.g. 0번, 1번 ind를 cx, mut   // 2번, 3번 ind를 cx, mut // ...
            for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
                if random.random() <= self.cxpb:
                    self.toolbox.mate(ind1, ind2)

                self.toolbox.mutate(ind1, indpb=self.mutpb)
                self.toolbox.mutate(ind2, indpb=self.mutpb)
                del ind1.fitness.values, ind2.fitness.values

            ##### 3.3. Evaluation
            # Evaluate the individuals with an invalid fitness
            print("\t Evaluation...")
            start_time = time.time()

            # fitness values (= accuracy, flops) 모음
            GA_history_list = []
            
            invalid_ind = [ind for ind in offspring]
            
            for idx, ind in enumerate(invalid_ind):
                fitness, ind_model = evaluate_v2(ind, args_train=self.args_train, stage_pool_path_list=self.stage_pool_path_list, data_path=self.data_path, log_file_name=self.log_file_name)
                # <= evaluate() returns  (-prec, flops), NN_model
                
                ind.fitness.values = fitness
                GA_history_list.append([ind, fitness])

            ## log 기록
            self.train_log[gen] = GA_history_list
            
            self.save_log()

            eval_time_for_one_generation = time.time() - start_time
            print("\t Evaluation ends (Time : %.3f)" % eval_time_for_one_generation)

            ##### Select the next generation population
            pop = self.toolbox.select(pop + offspring, self.pop_size)

            gen_time = time.time() - start_gen
            print('\t [gen_time: %.3fs]' % gen_time, gen, 'th generation is finished.')

            logging.info('\t Gen [%03d/%03d] -- evals: %03d, evals_time: %.4fs, gen_time: %.4fs' % (
                gen, self.ngen, len(invalid_ind), eval_time_for_one_generation, gen_time))


    ## Save Log
    def save_log(self):
        ## 필요한 log 추후 정리하여 추가 
        self.log['train_log'] = self.train_log

        with open(self.train_log_file_name, 'w', encoding='utf-8') as make_file:
            json.dump(self.log, make_file, ensure_ascii=False, indent='\t')

            
        
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--params', type=str, help='Parameter Json file')
    args = parser.parse_args()
    
    trainer = rwns_train(json_file=args.params)
#     trainer.create_toolbox()
    trainer.train()
    trainer.save_log()
